In [88]:
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Circle
from pathlib import Path

def main(out_dir=".", seed=42):
    rng = np.random.default_rng(seed)

    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    gif1 = out_dir / "coverage_step1_unit.gif"
    gif2 = out_dir / "coverage_step2_property.gif"
    gif3 = out_dir / "coverage_step3_theorem.gif"

    # --- Settings ---
    ax_lim = 1.0
    circle_radius = 0.4
    n_unit = 5
    n_prop = 40
    scale_prop = 0.9
    fps = 20
    freeze_time = 10  # seconds

    unit_points = np.array([
        [-0.3,  0.5],
        [-0.1,  0.2],
        [ 0.3,  0.2],
        [ 0.15, -0.2],
        [-0.2,  0.0],
    ])
    n_unit = min(n_unit, len(unit_points))
    prop_points = rng.uniform(-ax_lim, ax_lim, size=(n_prop, 2)) * scale_prop

    # --- Helper: consistent minimalist axes ---
    def setup_axes():
        fig, ax = plt.subplots(figsize=(5, 4.5), dpi=150)
        ax.set_xlim(-ax_lim, ax_lim)
        ax.set_ylim(-ax_lim, ax_lim)
        ax.set_aspect("equal", adjustable="box")

        # Hide frame and ticks
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(False)

        # Draw only x/y axes
        ax.axhline(0, color="black", linewidth=0.8)
        ax.axvline(0, color="black", linewidth=0.8)

        # Adjust to make space for legend
        fig.subplots_adjust(right=0.8)  # slightly wider
        return fig, ax

    def place_legend(ax):
        """Unified legend placement."""
        return ax.legend(
            loc="center left",
            bbox_to_anchor=(0.70, 1.07),
            frameon=False,
            borderaxespad=0.0,
        )

    # --- GIF 1: Unit tests ---
    freeze_frames = int(freeze_time * fps)
    frames1 = n_unit + 5 + freeze_frames
    fig1, ax1 = setup_axes()
    unit_scatter = ax1.scatter([], [], s=30, c="tab:blue", label="Unit tests")
    # place_legend(ax1)

    def init1():
        unit_scatter.set_offsets(np.empty((0, 2)))
        return (unit_scatter,)

    def update1(frame):
        k = min(frame, n_unit)
        unit_scatter.set_offsets(unit_points[:k])
        return (unit_scatter,)

    ani1 = animation.FuncAnimation(
        fig1, update1, init_func=init1,
        frames=frames1, interval=1000/fps, blit=True
    )
    ani1.save(gif1, writer=animation.PillowWriter(fps=fps, metadata={"loop": 1}))
    plt.close(fig1)

    # --- GIF 2: Add property-based tests ---
    frames2 = n_prop + 10 + freeze_frames
    fig2, ax2 = setup_axes()
    ax2.scatter(unit_points[:, 0], unit_points[:, 1], s=30, c="tab:blue", label="Unit tests")
    prop_scatter = ax2.scatter([], [], s=30, c="tab:orange", alpha=0.9, label="Property-based tests")
    # place_legend(ax2)

    def init2():
        prop_scatter.set_offsets(np.empty((0, 2)))
        return (prop_scatter,)

    def update2(frame):
        k = min(frame, n_prop)
        prop_scatter.set_offsets(prop_points[:k])
        return (prop_scatter,)

    ani2 = animation.FuncAnimation(
        fig2, update2, init_func=init2,
        frames=frames2, interval=1000/fps, blit=True
    )
    ani2.save(gif2, writer=animation.PillowWriter(fps=fps))
    plt.close(fig2)

    # --- GIF 3: Theorem-proving circle ---
    draw_frames = 20
    hold_frames = 20
    frames3 = draw_frames + hold_frames + freeze_frames
    theta = np.linspace(0, 2*np.pi, 720)
    circle_x = circle_radius * np.cos(theta)
    circle_y = circle_radius * np.sin(theta)

    fig3, ax3 = setup_axes()
    ax3.scatter(unit_points[:, 0], unit_points[:, 1], s=30, c="tab:blue", label="Unit test")
    ax3.scatter(prop_points[:, 0], prop_points[:, 1], s=30, c="tab:orange", alpha=0.9, label="Property-based test")
    (circle_line,) = ax3.plot([], [], linewidth=2.0, color="black", label="Theorem proving")
    # place_legend(ax3)

    fill_patch = Circle((0, 0), circle_radius, facecolor="green", alpha=0.25, edgecolor="none", visible=False)
    ax3.add_patch(fill_patch)

    def init3():
        circle_line.set_data([], [])
        fill_patch.set_visible(False)
        return (circle_line, fill_patch)

    def update3(frame):
        if frame < draw_frames:
            max_idx = int((frame + 1) / draw_frames * len(theta))
            circle_line.set_data(circle_x[:max_idx], circle_y[:max_idx])
        else:
            circle_line.set_data(circle_x, circle_y)
            fill_patch.set_visible(True)
        return (circle_line, fill_patch)

    ani3 = animation.FuncAnimation(
        fig3, update3, init_func=init3,
        frames=frames3, interval=1000/fps, blit=True
    )
    ani3.save(gif3, writer=animation.PillowWriter(fps=fps))
    plt.close(fig3)


In [87]:
main()